# Deciding Maximality

Non-uniform (hybrid) finite cellular automata (CAs) under null boundary condition
share a very special property which the classical finite CAs or the non-uniform finite
CAs under periodic boundary condition do not. This property is the maximality in
cycle length, which means, the presence of a cycle of length $2^n − 1$ in an n-cell binary
automaton. This property was first observed by Pries et. al. The maximal length
CAs are reversible CAs.

*A finite CA is a maximal length CA if all but one configuration of it
are in a single cycle.*

Hybrid rule vectors consisting of only rules 90 and 150 produces maximal length CA. This is a very generic algorithm to determine maximality, which takes exponential runtime. This is based upon [Exhaustive Search] and a experimental approach.

In [73]:
generate_binary_strings = lambda n: [format(i, f'0{n}b') for i in range(2**n)]

def decideMaximality(n_input,index_set):
    
    # the index_set takes the rule vector by only including the index of cells following rule 90
    # for example index_set = {0,2,4} means [90,150,90,150,90], index_set = {1,3,4} means [150,90,150,90,90]

    rule_90 = {}
    rule_150  = {}
    r_90 = f'{90:0{8}b}'
    r_150 = f'{150:0{8}b}'
    for i,key in enumerate(generate_binary_strings(3)[::-1]):
        rule_90[key] = r_90[i]
        rule_150[key] = r_150[i]
    

    current = f'{1:0{n_input}b}'             #started with ...00001

    rule_vector = [90 if i in index_set else 150 for i in range(n_input)]

    print("The rule vector is",rule_vector,"and the cell length is",n_input)
    
    have = current
    new = '0'*(n_input+1)
    periodicity = 0
    store = {}
    while have!=new:
        
        exec = '0' + current + '0'
        new = ''
        for i in range(len(exec)-2):
            if i in index_set:
                new += rule_90[exec[i:i+3]]
            if i not in index_set:
                new += rule_150[exec[i:i+3]]
        if new in store:
            
            if new!=f'{1:0{n_input}b}':
                return 'not cyclic thus not maximal'
                
        current = new
        store[new] = periodicity
        periodicity += 1
    else:
        if periodicity == 2**n_input-1:
            for key in store.keys():
                print(key)
            print("periodicity is",periodicity)
            return "maximal-length CA"
        else:
            return "Not a maximal_length CA"



In [74]:
decideMaximality(4,{0,2,3})

The rule vector is [90, 150, 90, 90] and the cell length is 4


'Not a maximal_length CA'

In [75]:
decideMaximality(4,{2})

The rule vector is [150, 150, 90, 150] and the cell length is 4
0011
0110
1011
1010
1001
1111
0100
1110
0111
1000
1100
0010
0101
1101
0001
periodicity is 15


'maximal-length CA'

In [76]:
decideMaximality(4,{2,3})

The rule vector is [150, 150, 90, 90] and the cell length is 4


'not cyclic thus not maximal'